# DDPLKO Moduł 4 - praca domowa - Quickdraw 10 class - regularyzacja

Twoim zadaniem w tym module będzie przygotowanie własnego modelu sieci neuronowej korzystając z regularyzacji.

Lista rzeczy które musi spełnić Twój model:
- [x] działać na wybranych przez Ciebie 10 klasach (bazuj na kodzie z modułu 3)
- [ ] liczba parametrów pomiędzy 100'000 a 200'000
- [ ] wykorzystane przynajmniej 2 sposoby walki z regularyzacją
- [ ] mieć wykonane co najmniej 4 zmiany w celu poprawy wyniku; zachowaj wszystkie iteracje (modyfikując model możesz dodać opcje w funkji, bądź skopiować klasę/funkcję, tak by było widać kolejne architektury)
- [ ] opisz co chcesz sprawdzić w kolejnych eksperymentach (np. sprawdzę czy Dropout pomaga i z jaką wartością drop ratio najbardziej)
- [ ] uzyskiwać lepsze `validation accuracy` niż w przypadku pierwszego modelu z poprzedniego modułu (im więcej punktów procentowych różnicy tym lepiej)

Zwizualizuj proszę:
- [ ] historie treningów (wystarczy Val acc, ale train acc czy lossy też mogą być)
- [ ] zależność: liczba parametrów - val acc

Możesz (czyli opcjonalne rzeczy):
- pracować na zmniejszonym zbiorze, by dobrać wartość parametrów
- np. zastosować dropout, pooling i early stopping
- zastosować TF2 - Keras / PyTorcha czy PL (Pytorch Lightning)
- dodać LR scheduler do swojego treningu (i sprawdzić czy to poprawiło wynik)
- zwizualizować dodatkowo:
  - confussion matrix
  - błędne przypadki

Warto:
- zmieniać 1 parametr między eksperymentami (szczególnie trudne gdy się już nabierze wyczucia)

In [1]:
import urllib
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split

class_names = ["airplane", "banana", "cookie", "diamond", "dog", "hot air balloon", "knife" ,"parachute", "scissors", "wine glass"]

# wczytanie danych

data_folder = "../data/quickdraw/"

for name in class_names:
    url = 'https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/%s.npy'%name
    file_name = data_folder + url.split('/')[-1].split('?')[0]

    url = url.replace(' ','%20')

    if not os.path.isfile(file_name):
        print(url, '==>', file_name)
        urllib.request.urlretrieve(url, file_name)

data = []
for name in class_names:
    file_name = data_folder + name + '.npy'
    data.append(np.load(file_name, fix_imports=True, allow_pickle=True))
    print('%-15s'%name,type(data[-1]))

X = np.concatenate(data).reshape(-1,28,28,1)
y = np.concatenate([np.full(d.shape[0], i) for i, d in enumerate(data)])

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=12, stratify=y)

airplane        <class 'numpy.ndarray'>
banana          <class 'numpy.ndarray'>
cookie          <class 'numpy.ndarray'>
diamond         <class 'numpy.ndarray'>
dog             <class 'numpy.ndarray'>
hot air balloon <class 'numpy.ndarray'>
knife           <class 'numpy.ndarray'>
parachute       <class 'numpy.ndarray'>
scissors        <class 'numpy.ndarray'>
wine glass      <class 'numpy.ndarray'>


In [12]:
import torch
from torch.nn import functional as F
from torch import nn
from torch.utils.data import TensorDataset, DataLoader

from torchvision import datasets, transforms

import lightning.pytorch as pl
from lightning.pytorch.loggers import TensorBoardLogger

from torchmetrics.functional.classification.accuracy import accuracy

class QuickDrawCNN_PL(pl.LightningModule):
    def __init__(self, X_train,y_train,X_val,y_val, batch_size):
        super().__init__()

        self.X_train = torch.FloatTensor(X_train).permute(0, 3, 1, 2)
        self.X_val = torch.FloatTensor(X_val).permute(0, 3, 1, 2)
        self.y_train = torch.LongTensor(y_train)
        self.y_val = torch.LongTensor(y_val)
        self.train_dataset = TensorDataset(self.X_train, self.y_train)
        self.val_dataset = TensorDataset(self.X_val, self.y_val)

        self.batch_size = batch_size

        ####################
        ### Don't chagne ###
        assert type(self.X_train)==torch.Tensor
        assert self.X_train.shape==torch.Size([len(X_train), 1, 28, 28])
        assert self.X_train.dtype==torch.float32, "Typ X_train niepoprawny"

        assert type(self.y_train)==torch.Tensor
        assert self.y_train.shape==torch.Size([len(X_train)])
        assert self.y_train.dtype==torch.int64, "Typ y_train niepoprawny"

        assert type(self.X_val)==torch.Tensor
        assert self.X_val.shape==torch.Size([len(X_val), 1, 28, 28])
        assert self.X_val.dtype==torch.float32, "Typ X_val niepoprawny"

        assert type(self.y_val)==torch.Tensor
        assert self.y_val.shape==torch.Size([len(y_val)])
        assert self.y_val.dtype==torch.int64, "Typ y_val niepoprawny"
        ### Don't chagne ###
        ####################


        self.num_classes = 10
        self.dims = (1, 28, 28)
        channels, width, height = self.dims

        self.model = nn.Sequential(
            nn.Conv2d(channels, 32, 3),
            nn.ReLU(),
            nn.MaxPool2d(2),
            #nn.AdaptiveAvgPool2d((1, 1)),
            nn.Conv2d(32, 64, 3),
            nn.ReLU(),
            nn.MaxPool2d(2),
            #nn.AdaptiveAvgPool2d((1, 1)),
            nn.Conv2d(64, 256, 3),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(256, 32),
            nn.Linear(32, self.num_classes), 
        )

        ####################
        ### Don't chagne ###
        assert any(['Conv2d' in str(_) for _ in self.model]), "Zastosuj przynajmniej jedną warstwę Conv2d"
        assert len([_ for _ in self.model if 'Conv2d' in str(_)])==len([_ for _ in self.model if 'ReLU' in str(_)]), "Po każdej warstwie Conv2d zastosuj funkcję aktywacji ReLU"
        ### Don't chagne ###
        ####################
    def forward(self, x):
        x = self.model(x)
        return F.log_softmax(x, dim=1)

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.nll_loss(logits, y)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-3, weight_decay=1e-5)

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.nll_loss(logits, y)
        preds = torch.argmax(logits, dim=1)
        acc = accuracy(preds, y, task="multiclass", num_classes=self.num_classes)

        # Calling self.log will surface up scalars for you in TensorBoard
        self.log('val_loss', loss, prog_bar=True)
        self.log('val_acc', acc, prog_bar=True)
        return loss

    def test_step(self, batch, batch_idx):
        # Here we just reuse the validation_step for testing
        return self.validation_step(batch, batch_idx)

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, num_workers=16)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size, num_workers=16)

In [13]:
# check model summary 
model = QuickDrawCNN_PL(X_train,y_train,X_val,y_val, batch_size=32)
print(model)

# check number of parameters
num_of_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print('Number of parameters:', num_of_params)

QuickDrawCNN_PL(
  (model): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(64, 256, kernel_size=(3, 3), stride=(1, 1))
    (7): ReLU()
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): Flatten(start_dim=1, end_dim=-1)
    (10): Linear(in_features=256, out_features=32, bias=True)
    (11): Linear(in_features=32, out_features=10, bias=True)
  )
)
Number of parameters: 175082


In [10]:
assert num_of_params > 100_000, "Za mało parametrów"
assert num_of_params < 200_000, "Za dużo parametrów"

# Trening

In [17]:
# You are using a CUDA device ('NVIDIA GeForce RTX 4070 Ti') that has Tensor Cores. 
# To properly utilize them, you should set 
# `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance
torch.set_float32_matmul_precision('medium')

model = QuickDrawCNN_PL(X_train,y_train,X_val,y_val, 128*4)
logger = TensorBoardLogger("lightning_logs", name="modul_3")

trainer = pl.Trainer(
    max_epochs=10, 
    precision=16, 
    accelerator="gpu",
    logger=logger,
)

trainer.fit(model)

C:\Users\Wiktor\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\lightning\fabric\connector.py:554: UserWarning: 16 is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
  rank_zero_warn(
Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type       | Params
-------------------------------------
0 | model | Sequential | 175 K 
-------------------------------------
175 K     Trainable params
0         Non-trainable params
175 K     Total params
0.700     Total estimated model params size (MB)


Epoch 9: 100%|██████████| 2433/2433 [00:48<00:00, 49.93it/s, v_num=8, val_loss=0.222, val_acc=0.933]

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 2433/2433 [00:48<00:00, 49.92it/s, v_num=8, val_loss=0.222, val_acc=0.933]


In [18]:
%load_ext tensorboard
%tensorboard --logdir lightning_logs/

# Wyślij rozwiązanie
Możesz skorzystać z jednego z poniższych sposobów:
**mailem na specjalny adres** ze strony pracy domowej w panelu programu prześlij jedno z poniższych:
- notebooka (jeżeli plik ma mniej niż np. 10MB)
- notebooka w zipie
- link do Colaba (udostępniony)
- link do pliku przez GDrive/Dropboxa/WeTransfer/...
- pdfa (poprzez download as pdf)
- jako plik w repozytorium na np. GitHubie, by budować swoje portfolio (wtedy uważaj na wielkość pliku, najlepiej kilka MB, Max 25MB)

Najlepiej, by w notebooku było widać wyniki uruchomienia komórek, chyba, że przez nie plik będzie mieć 100+MB wtedy najlepiej Colab lub jakieś przemyślenie co poszło nie tak (zbyt dużo dużych zdjęć wyświetlonych w komórkach).

## Co otrzymasz?
Informację zwrotną z ewentualnymi sugestiami, komentarzami.